<a href="https://colab.research.google.com/github/connoro28/LocationApp/blob/main/car_classification6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PyTorch core libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Torchvision for computer vision tasks
import torchvision
from torchvision import transforms, models

# For data handling and splitting
import pandas as pd
from sklearn.model_selection import train_test_split

# For image loading and handling
from PIL import Image

# Other useful libraries
import os
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the zip file in your Google Drive
zip_path = '/content/drive/MyDrive/car_dataset_colab/all_images_flat.zip'

# Path where you want to extract the images in the Colab environment
extract_path = '/content/images/'

print("Starting to unzip images...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Unzipping complete!")

Starting to unzip images...
Unzipping complete!


In [ ]:
import pandas as pd
import os
import ast

# --- 1. Define ABSOLUTE paths for Colab ---
# Path to your project folder in Google Drive
drive_project_path = r'/content/drive/MyDrive/car_dataset_colab'

# Path to the Excel file in Drive
excel_path = os.path.join(drive_project_path, 'car_data_full.xlsx')

# Path to the UNZIPPED images on Colab's fast local storage
base_image_dir = os.path.join(extract_path, 'all_images_flat')

# --- 2. Load and Restructure the DataFrame ---
# (The rest of this cell remains the same as your working version)
df = pd.read_excel(excel_path)
df['image_file_names'] = df['image_file_names'].apply(ast.literal_eval)
df_exploded = df.explode('image_file_names').reset_index(drop=True)
print("DataFrame successfully restructured.")

# --- 3. Create the correct Full Image Path ---
def get_prefix_from_path(path_str):
    parts = str(path_str).replace('\\', '/').split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return ""

df_exploded['prefix'] = df_exploded['dir_path'].apply(get_prefix_from_path)
df_exploded['image_file_names'] = df_exploded['image_file_names'].astype(str)
df_exploded['new_filename'] = df_exploded['prefix'] + '_' + df_exploded['image_file_names']
df_exploded['Full Image Path'] = df_exploded['new_filename'].apply(
    lambda filename: os.path.join(base_image_dir, filename)
)

# --- 4. Verification and Sanity Check ---
print("\nVerifying the final, Colab paths...")
first_path = df_exploded['Full Image Path'].iloc[0]
if os.path.exists(first_path):
    print(f"\nSUCCESS: The first file was found at: {first_path}")
else:
    print(f"\nFAILURE: The first file was NOT found at: {first_path}")

DataFrame successfully restructured.

Verifying the final, Colab paths...

SUCCESS: The first file was found at: /content/images/all_images_flat/AC__428_Convertible_19661971_ac-428-convertible-1966-7054_1.jpg


In [ ]:
# --- First, filter the main DataFrame ---
brand_counts = df_exploded['brand'].value_counts()
brands_to_keep = brand_counts[brand_counts > 1].index
df_filtered = df_exploded[df_exploded['brand'].isin(brands_to_keep)].copy()

print(f"Original number of rows: {len(df_exploded)}")
print(f"Number of rows after initial filtering: {len(df_filtered)}")
print("-" * 30)

# --- Perform the first split ---

features = df_filtered['Full Image Path']
labels = df_filtered['brand']

X_train, X_temp, y_train, y_temp = train_test_split(
    features, labels, test_size=0.3, random_state=42, stratify=labels)


# --- FIX: Filter the temporary set AGAIN before the second split ---
temp_brand_counts = y_temp.value_counts()
brands_to_keep_in_temp = temp_brand_counts[temp_brand_counts > 1].index

# Keep only the rows in X_temp and y_temp that correspond to the brands we want to keep
X_temp_filtered = X_temp[y_temp.isin(brands_to_keep_in_temp)]
y_temp_filtered = y_temp[y_temp.isin(brands_to_keep_in_temp)]


# --- Now, perform the second split on the filtered temporary data ---
X_val, X_test, y_val, y_test = train_test_split(
    X_temp_filtered, y_temp_filtered, test_size=0.5, random_state=42, stratify=y_temp_filtered)


# --- Final verification ---
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Testing samples: {len(X_test)}")

Original number of rows: 194727
Number of rows after initial filtering: 194727
------------------------------
Training samples: 136308
Validation samples: 29209
Testing samples: 29210


In [ ]:
# Define the image transformations for the training set
# Includes data augmentation
train_transforms = transforms.Compose([
    transforms.Resize(256),             # Resize the smaller edge to 256
    transforms.RandomResizedCrop(224),  # Crop a random 224x224 part
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize(               # Normalize with ImageNet's mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Define the image transformations for the validation and test sets
# No data augmentation
val_test_transforms = transforms.Compose([
    transforms.Resize(256),             # Resize the smaller edge to 256
    transforms.CenterCrop(224),         # Crop the center 224x224 part
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize(               # Normalize with ImageNet's mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Image transformation pipelines defined successfully!")

Image transformation pipelines defined successfully!


In [ ]:
# --- 1. Create a mapping from brand names to integer labels ---
# Get a list of unique brand names from the original filtered labels
all_labels = df_filtered['brand'].unique()
# Create the mapping
class_to_idx = {label: i for i, label in enumerate(all_labels)}
# Also create the reverse mapping to get the name back from a number
idx_to_class = {i: label for label, i in class_to_idx.items()}

# --- 2. Define the Custom Dataset Class ---
class CarDataset(Dataset):
    def __init__(self, image_paths, labels, class_to_idx, transform=None):
        """
        Args:
            image_paths (pandas.Series): A pandas Series of image file paths.
            labels (pandas.Series): A pandas Series of corresponding labels.
            class_to_idx (dict): A dictionary mapping class names to indices.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # We need to use .values to get the underlying numpy arrays
        self.image_paths = image_paths.values
        self.labels = labels.values
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        # This returns the total number of samples in the dataset
        return len(self.image_paths)

    def __getitem__(self, idx):
        # This method gets one sample from the dataset
        # 1. Get the image path
        img_path = self.image_paths[idx]

        # 2. Open the image using Pillow
        # We use a try-except block to handle potentially corrupt images
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            #print(f"Warning: Could not load image {img_path}. Skipping. Error: {e}")
            # Return the first image and label as a fallback
            img_path = self.image_paths[0]
            image = Image.open(img_path).convert('RGB')

        # 3. Get the corresponding text label and convert it to its integer index
        label_name = self.labels[idx]
        label_idx = self.class_to_idx[label_name]

        # 4. Apply transformations, if any
        if self.transform:
            image = self.transform(image)

        return image, label_idx

print("CarDataset class defined successfully!")

CarDataset class defined successfully!


In [ ]:
# --- 1. Create the Dataset objects ---

# Training dataset with data augmentation
train_dataset = CarDataset(
    image_paths=X_train,
    labels=y_train,
    class_to_idx=class_to_idx,
    transform=train_transforms
)

# Validation dataset with standard transformations
val_dataset = CarDataset(
    image_paths=X_val,
    labels=y_val,
    class_to_idx=class_to_idx,
    transform=val_test_transforms
)

# Test dataset with standard transformations
test_dataset = CarDataset(
    image_paths=X_test,
    labels=y_test,
    class_to_idx=class_to_idx,
    transform=val_test_transforms
)


# --- 2. Create the DataLoader objects (UPDATED FOR SPEED) ---

# Define the batch size
batch_size = 128

# Training DataLoader (shuffled to ensure random batches each epoch)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,  # <-- THE FIX: Use 2 parallel workers to load data
    pin_memory=True # <-- Helps speed up data transfer to the GPU
)

# Validation DataLoader (no need to shuffle)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,  # <-- THE FIX
    pin_memory=True # <-- THE FIX
)

# Test DataLoader (no need to shuffle)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,  # <-- THE FIX
    pin_memory=True # <-- THE FIX
)

print("Optimized DataLoaders created successfully!")
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")


Optimized DataLoaders created successfully!
Number of batches in train_loader: 1065
Number of batches in val_loader: 229


In [ ]:
# --- 1. Set up the device (use GPU if available, otherwise CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. Load the pre-trained model ---
# We use ResNet50, with weights pre-trained on the ImageNet dataset
model = models.resnet50(weights='IMAGENET1K_V1')

# --- 3. Freeze the parameters of the base model ---
# We don't want to change the pre-learned features during initial training
for param in model.parameters():
    param.requires_grad = False

# --- 4. Replace the final classifier layer ---
# Get the number of input features for the classifier
num_ftrs = model.fc.in_features

# Get the number of classes (car brands) from our dataset
num_classes = len(class_to_idx)

# Create a new, unfrozen fully-connected layer to replace the old one
# This new layer's weights WILL be updated during training
model.fc = nn.Linear(num_ftrs, num_classes)

# --- 5. Move the model to the selected device ---
model = model.to(device)

# (Optional) Print the model architecture to see the new final layer
print("\nModel architecture updated successfully!")
#print(model) # Uncomment this line if you want to see the full architecture

Using device: cuda

Model architecture updated successfully!


In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
# We only want to train the parameters of the new classifier layer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

print("Loss function and optimizer defined successfully.")

Loss function and optimizer defined successfully.


In [ ]:
import time

# A function to calculate accuracy
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred.data, 1)
    total = y_true.size(0)
    correct = (predicted == y_true).sum().item()
    return correct / total

# --- AMP: Initialize the Gradient Scaler ---
scaler = torch.cuda.amp.GradScaler()

# Define the number of epochs
num_epochs = 10

print("Starting training with Automatic Mixed Precision...")
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_acc = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # --- AMP: Wrap the forward pass in autocast ---
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # --- AMP: Scale the loss and call backward() ---
        # Instead of loss.backward()
        scaler.scale(loss).backward()

        # --- AMP: Unscales gradients and calls optimizer.step() ---
        # Instead of optimizer.step()
        scaler.step(optimizer)

        # --- AMP: Update the scale for next iteration ---
        scaler.update()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Gather statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += calculate_accuracy(outputs, labels) * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_acc / len(train_dataset)

    # --- Validation Phase (no changes needed here) ---
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_acc += calculate_accuracy(outputs, labels) * inputs.size(0)

    epoch_val_loss = val_loss / len(val_dataset)
    epoch_val_acc = val_acc / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")

end_time = time.time()
print(f"\nTraining finished in {(end_time - start_time)/60:.2f} minutes.")

Starting training with Automatic Mixed Precision...


/tmp/ipython-input-23-4196050525.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch 1/10 | Train Loss: 3.4887 | Train Acc: 0.1433 | Val Loss: 3.3658 | Val Acc: 0.1821
Epoch 2/10 | Train Loss: 3.2975 | Train Acc: 0.1773 | Val Loss: 3.3290 | Val Acc: 0.1903
Epoch 3/10 | Train Loss: 3.2402 | Train Acc: 0.1880 | Val Loss: 3.2805 | Val Acc: 0.1981
Epoch 4/10 | Train Loss: 3.1949 | Train Acc: 0.1975 | Val Loss: 3.3042 | Val Acc: 0.2050


KeyboardInterrupt: 